In [1]:
import numpy as np # linear algebra
import polars as pl # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [2]:
w_teams = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/WTeams.csv").collect()
m_teams = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/MTeams.csv").collect()
m_compact_results = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv").collect()
w_compact_results = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv").collect()
m_rankings = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/MMasseyOrdinals.csv").collect()

w_seeds = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySeeds.csv").collect()
m_seeds = pl.scan_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv").collect()

submission_stage1 = pl.scan_csv ("/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage1.csv").collect()
submission_stage2 = pl.scan_csv ("/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage2.csv").collect()

m_truth = pl.scan_csv ("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv").collect()
w_truth = pl.scan_csv ("/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv").collect()

In [3]:
def extract_seeds (seeds : pl.DataFrame) -> pl.DataFrame :
    
    result = seeds.with_columns (pl.col("Seed").str.head(1).alias("Conference"), 
                               pl.col("Seed").str.slice(1,2).cast(pl.UInt8).alias("C_Seed"))
    return result.drop("Seed")

w_seeds_Season = extract_seeds (w_seeds) 
m_seeds_Season = extract_seeds (m_seeds) 

In [4]:
print (m_rankings.head())
def extract_rankings (rankings : pl.DataFrame) -> pl.DataFrame :
    rankings  = rankings.with_columns ((pl.col("Season").cast (pl.String) + "_" + pl.col("TeamID").cast(pl.String)).alias ("Season_Team"))

    last_ranking_day  = rankings.group_by(["Season", "SystemName"]).agg(pl.col("RankingDayNum").max()).sort("Season")

    rankings_last = rankings.join (last_ranking_day, how = "inner", on = ["Season", "SystemName", "RankingDayNum"])


    team_rankings = rankings_last.pivot("SystemName", index="Season_Team", values="OrdinalRank")

    team_rankings = team_rankings.with_columns (pl.col("Season_Team").str.split("_").list.first().cast (pl.Int64).alias ("Season"),
                                               pl.col("Season_Team").str.split("_").list.last().cast (pl.Int64).alias ("TeamID"))

    return team_rankings.drop("Season_Team")

m_team_rankings = extract_rankings (m_rankings)
print (m_team_rankings.head())

shape: (5, 5)
┌────────┬───────────────┬────────────┬────────┬─────────────┐
│ Season ┆ RankingDayNum ┆ SystemName ┆ TeamID ┆ OrdinalRank │
│ ---    ┆ ---           ┆ ---        ┆ ---    ┆ ---         │
│ i64    ┆ i64           ┆ str        ┆ i64    ┆ i64         │
╞════════╪═══════════════╪════════════╪════════╪═════════════╡
│ 2003   ┆ 35            ┆ SEL        ┆ 1102   ┆ 159         │
│ 2003   ┆ 35            ┆ SEL        ┆ 1103   ┆ 229         │
│ 2003   ┆ 35            ┆ SEL        ┆ 1104   ┆ 12          │
│ 2003   ┆ 35            ┆ SEL        ┆ 1105   ┆ 314         │
│ 2003   ┆ 35            ┆ SEL        ┆ 1106   ┆ 260         │
└────────┴───────────────┴────────────┴────────┴─────────────┘
shape: (5, 194)
┌─────┬─────┬──────┬─────┬───┬──────┬──────┬────────┬────────┐
│ GC  ┆ MIC ┆ AP   ┆ ARG ┆ … ┆ BMN  ┆ PAC  ┆ Season ┆ TeamID │
│ --- ┆ --- ┆ ---  ┆ --- ┆   ┆ ---  ┆ ---  ┆ ---    ┆ ---    │
│ i64 ┆ i64 ┆ i64  ┆ i64 ┆   ┆ i64  ┆ i64  ┆ i64    ┆ i64    │
╞═════╪═════╪══════╪═════

In [5]:
w_teams.head()

TeamID,TeamName
i64,str
3101,"""Abilene Chr"""
3102,"""Air Force"""
3103,"""Akron"""
3104,"""Alabama"""
3105,"""Alabama A&M"""


In [6]:
display (submission_stage1.sort ("ID"))
display (submission_stage2.sort ("ID"))

ID,Pred
str,f64
"""2021_1101_1102""",0.5
"""2021_1101_1103""",0.5
"""2021_1101_1104""",0.5
"""2021_1101_1105""",0.5
"""2021_1101_1106""",0.5
…,…
"""2024_3475_3477""",0.5
"""2024_3475_3478""",0.5
"""2024_3476_3477""",0.5


ID,Pred
str,f64
"""2025_1101_1102""",0.5
"""2025_1101_1103""",0.5
"""2025_1101_1104""",0.5
"""2025_1101_1105""",0.5
"""2025_1101_1106""",0.5
…,…
"""2025_3477_3479""",0.5
"""2025_3477_3480""",0.5
"""2025_3478_3479""",0.5


In [7]:
w_compact_results.get_column("Season").unique()

Season
i64
1998
1999
2000
2001
2002
…
2021
2022
2023


In [8]:
m_compact_results.filter (pl.col("Season") == 2021).describe()

statistic,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
str,f64,f64,f64,f64,f64,f64,str,f64
"""count""",3855.0,3855.0,3855.0,3855.0,3855.0,3855.0,"""3855""",3855.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0
"""mean""",2021.0,79.274189,1290.841245,76.808301,1288.496757,64.807004,null,0.067445
"""std""",0.0,31.420038,105.990924,10.516196,104.52898,10.284112,null,0.303271
"""min""",2021.0,23.0,1101.0,45.0,1101.0,30.0,"""A""",0.0
"""25%""",2021.0,51.0,1202.0,69.0,1199.0,58.0,null,0.0
"""50%""",2021.0,82.0,1288.0,76.0,1291.0,65.0,null,0.0
"""75%""",2021.0,107.0,1385.0,83.0,1379.0,71.0,null,0.0
"""max""",2021.0,132.0,1471.0,142.0,1471.0,120.0,"""N""",4.0


In [9]:
w_truth

Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
i64,i64,i64,i64,i64,i64,str,i64
1998,137,3104,94,3422,46,"""H""",0
1998,137,3112,75,3365,63,"""H""",0
1998,137,3163,93,3193,52,"""H""",0
1998,137,3198,59,3266,45,"""H""",0
1998,137,3203,74,3208,72,"""A""",0
…,…,…,…,…,…,…,…
2024,147,3163,80,3425,73,"""A""",0
2024,147,3234,94,3261,87,"""H""",0
2024,151,3234,71,3163,69,"""N""",0


In [10]:
def win_loss_record (teams : pl.DataFrame) -> pl.DataFrame :
    
    result = teams.select ("TeamID")
    base_file =  w_compact_results if (teams.get_column("TeamID").min() > 2000) else m_compact_results
    # season_results = base_file.filter (pl.col("Season") == season)
    
        
    wins = base_file.group_by (["Season","WTeamID"]).len()
    losses = base_file.group_by (["Season","LTeamID"]).len()
    wscores =  base_file.group_by (["Season","WTeamID"]).agg (pl.col("WScore").sum())
    lscores = base_file.group_by (["Season","LTeamID"]).agg (pl.col("LScore").sum())
    opp_wscores = base_file.group_by (["Season","WTeamID"]).agg (pl.col("LScore").sum())
    opp_lscores = base_file.group_by (["Season","LTeamID"]).agg (pl.col("WScore").sum())
    
    result = result.join (wins, how = "left", left_on = "TeamID", right_on = "WTeamID")
    result = result.rename ({"len" : "games won"})
    result = result.join (losses, how = "left", left_on = ["TeamID", "Season"], right_on = ["LTeamID", "Season"])
    result = result.rename ({"len" : "games lost"})
    result = result.join (wscores, how = "left", left_on = ["TeamID", "Season"], right_on = ["WTeamID", "Season"])
    result = result.rename ({"WScore" : "points in win"})
    result = result.join (lscores, how = "left", left_on = ["TeamID", "Season"], right_on = ["LTeamID", "Season"])
    result = result.rename ({"LScore" : "points in loss"})
    result = result.join (opp_wscores, how = "left", left_on = ["TeamID", "Season"], right_on = ["WTeamID", "Season"])
    result = result.rename ({"LScore" : "dev points in win"})
    result = result.join (opp_lscores, how = "left", left_on = ["TeamID", "Season"], right_on = ["LTeamID", "Season"])
    result = result.rename ({"WScore" : "dev points in loss"})
    result = result.with_columns ((pl.col("games won") + pl.col("games lost")).alias ("total games"), 
                                  (pl.col("points in win") + pl.col("points in loss")).alias ("total points"), 
                                  (pl.col("dev points in win") + pl.col("dev points in loss")).alias ("total dev points"))
    
    return result.fill_null(0)

male_team_results = win_loss_record (m_teams)

display (male_team_results)

female_team_results = win_loss_record (w_teams)

display (female_team_results)

TeamID,Season,games won,games lost,points in win,points in loss,dev points in win,dev points in loss,total games,total points,total dev points
i64,i64,u32,u32,i64,i64,i64,i64,u32,i64,i64
1101,2023,9,17,709,1142,607,1340,26,1851,1947
1101,2025,9,13,684,793,581,998,22,1477,1579
1101,2014,2,19,160,1166,153,1498,21,1326,1651
1101,2020,16,11,1220,745,1020,837,27,1965,1857
1101,2017,9,16,669,1028,627,1189,25,1697,1816
…,…,…,…,…,…,…,…,…,…,…
1477,2025,3,24,210,1516,185,1848,27,1726,2033
1478,2024,12,17,948,1081,754,1319,29,2029,2073
1478,2025,7,18,530,1270,484,1546,25,1800,2030


TeamID,Season,games won,games lost,points in win,points in loss,dev points in win,dev points in loss,total games,total points,total dev points
i64,i64,u32,u32,i64,i64,i64,i64,u32,i64,i64
3101,2024,11,16,866,999,674,1146,27,1865,1820
3101,2022,13,13,999,845,808,1012,26,1844,1820
3101,2014,11,12,863,747,728,880,23,1610,1608
3101,2015,13,12,970,656,763,803,25,1626,1566
3101,2018,12,14,927,787,784,1007,26,1714,1791
…,…,…,…,…,…,…,…,…,…,…
3477,2023,13,19,952,1116,815,1413,32,2068,2228
3478,2024,18,14,1151,674,918,1010,32,1825,1928
3478,2025,4,20,268,986,248,1468,24,1254,1716


In [11]:
def create_train (df,truth :pl.DataFrame) -> pl.DataFrame :
    
    truth_is_female = (truth.get_column ("WTeamID").min() > 2000)
    
    result = df.with_columns (pl.col("ID").str.split ("_").list.get(0).cast(pl.Int64).alias ("Season"),
                             pl.col("ID").str.split ("_").list.get(1).cast(pl.Int64).alias ("Team1"),
                             pl.col("ID").str.split ("_").list.get(2).cast(pl.Int64).alias ("Team2"))
   
    if truth_is_female :
        result = result.filter (pl.col('Team1') > 2000)
    else :
        result = result.filter (pl.col('Team1') < 2000)
    
    result = result.join (truth, how = "left", left_on = ["Season", "Team1", "Team2"], right_on = ["Season", "WTeamID", "LTeamID"])
    result = result.join (truth, how = "left", left_on = ["Season", "Team1", "Team2"], right_on = ["Season", "LTeamID", "WTeamID"])
   
   
    result = result.with_columns (pl.when (pl.col("WScore") > 0 ).then (pl.lit(1)).otherwise (
                                  pl.when (pl.col("WScore_right") > 0 ).then (pl.lit(0)).otherwise (
                                           pl.lit(0.5))).alias ("truth")
                                  
    )
    

    return result.drop ('Pred') 
    

In [12]:
male_training = create_train (submission_stage1, m_truth).filter ((pl.col("truth") ==1) | (pl.col("truth") ==0))
female_training = create_train (submission_stage1, w_truth).filter ((pl.col("truth") ==1) | (pl.col("truth") ==0))

male_submission = create_train (submission_stage2, m_truth)
female_submission = create_train (submission_stage2, w_truth)

print (f" total size {male_training.shape  }   ")
print (f" total size {male_submission.shape }   ")

 total size (267, 15)   
 total size (66066, 15)   


In [13]:
male_training = male_training.join (male_team_results, how = "left", left_on = ["Season","Team1"], right_on = ["Season","TeamID"])
male_training = male_training.join (male_team_results, how = "left", left_on = ["Season","Team2"], right_on = ["Season","TeamID"])

male_training = male_training.join (m_seeds_Season, how = "left", left_on = ["Season","Team1"], right_on = ["Season","TeamID"])
male_training = male_training.join (m_seeds_Season, how = "left", left_on = ["Season","Team2"], right_on = ["Season","TeamID"])
#male_training = male_training.join (m_team_rankings, how = "left", left_on = ["Season","Team1"], right_on = ["Season","TeamID"])
#male_training = male_training.join (m_team_rankings, how = "left", left_on = ["Season","Team2"], right_on = ["Season","TeamID"])

display (male_training)

print (male_training.schema)

female_training = female_training.join (female_team_results, how = "left", left_on = ["Season", "Team1"], right_on = ["Season","TeamID"])
female_training = female_training.join (female_team_results, how = "left", left_on = ["Season", "Team2"], right_on = ["Season","TeamID"])

female_training = female_training.join (w_seeds_Season, how = "left", left_on = ["Season", "Team1"], right_on = ["Season","TeamID"])
female_training = female_training.join (w_seeds_Season, how = "left", left_on = ["Season", "Team2"], right_on = ["Season","TeamID"])

display (female_training)
# X_train = pl.concat ([male_training, female_training], how = "vertical_relaxed").sample (fraction = 1, shuffle = True)

# X_train


ID,Season,Team1,Team2,DayNum,WScore,LScore,WLoc,NumOT,DayNum_right,WScore_right,LScore_right,WLoc_right,NumOT_right,truth,games won,games lost,points in win,points in loss,dev points in win,dev points in loss,total games,total points,total dev points,games won_right,games lost_right,points in win_right,points in loss_right,dev points in win_right,dev points in loss_right,total games_right,total points_right,total dev points_right,Conference,C_Seed,Conference_right,C_Seed_right
str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,i64,f64,u32,u32,i64,i64,i64,i64,u32,i64,i64,u32,u32,i64,i64,i64,i64,u32,i64,i64,str,u8,str,u8
"""2021_1101_1400""",2021,1101,1400,138,53,52,"""N""",0,null,null,null,null,null,1.0,19,4,1500,255,1136,284,23,1755,1420,19,7,1459,497,1252,537,26,1956,1789,"""W""",14,"""W""",3
"""2021_1101_1417""",2021,1101,1417,null,null,null,null,null,140,67,47,"""N""",0,0.0,19,4,1500,255,1136,284,23,1755,1420,17,9,1296,598,1112,669,26,1894,1781,"""W""",14,"""W""",11
"""2021_1104_1233""",2021,1104,1233,138,68,55,"""N""",0,null,null,null,null,null,1.0,24,6,2004,383,1659,434,30,2387,2093,12,5,900,328,734,387,17,1228,1121,"""W""",2,"""W""",15
"""2021_1104_1268""",2021,1104,1268,140,96,77,"""N""",0,null,null,null,null,null,1.0,24,6,2004,383,1659,434,30,2387,2093,15,13,1104,791,895,931,28,1895,1826,"""W""",2,"""W""",10
"""2021_1104_1417""",2021,1104,1417,null,null,null,null,null,146,88,78,"""N""",1,0.0,24,6,2004,383,1659,434,30,2387,2093,17,9,1296,598,1112,669,26,1894,1781,"""W""",2,"""W""",11
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024_1361_1412""",2024,1361,1412,137,69,65,"""N""",0,null,null,null,null,null,1.0,22,10,1704,670,1421,734,32,2374,2155,22,11,1761,793,1591,911,33,2554,2502,"""W""",5,"""W""",12
"""2024_1361_1463""",2024,1361,1463,139,85,57,"""N""",0,null,null,null,null,null,1.0,22,10,1704,670,1421,734,32,2374,2155,20,9,1524,609,1301,675,29,2133,1976,"""W""",5,"""W""",13
"""2024_1389_1397""",2024,1389,1397,null,null,null,null,null,136,83,49,"""N""",0,0.0,18,13,1229,789,1044,916,31,2018,1960,24,8,1987,556,1549,623,32,2543,2172,"""Y""",15,"""Y""",2


Schema([('ID', String), ('Season', Int64), ('Team1', Int64), ('Team2', Int64), ('DayNum', Int64), ('WScore', Int64), ('LScore', Int64), ('WLoc', String), ('NumOT', Int64), ('DayNum_right', Int64), ('WScore_right', Int64), ('LScore_right', Int64), ('WLoc_right', String), ('NumOT_right', Int64), ('truth', Float64), ('games won', UInt32), ('games lost', UInt32), ('points in win', Int64), ('points in loss', Int64), ('dev points in win', Int64), ('dev points in loss', Int64), ('total games', UInt32), ('total points', Int64), ('total dev points', Int64), ('games won_right', UInt32), ('games lost_right', UInt32), ('points in win_right', Int64), ('points in loss_right', Int64), ('dev points in win_right', Int64), ('dev points in loss_right', Int64), ('total games_right', UInt32), ('total points_right', Int64), ('total dev points_right', Int64), ('Conference', String), ('C_Seed', UInt8), ('Conference_right', String), ('C_Seed_right', UInt8)])


ID,Season,Team1,Team2,DayNum,WScore,LScore,WLoc,NumOT,DayNum_right,WScore_right,LScore_right,WLoc_right,NumOT_right,truth,games won,games lost,points in win,points in loss,dev points in win,dev points in loss,total games,total points,total dev points,games won_right,games lost_right,points in win_right,points in loss_right,dev points in win_right,dev points in loss_right,total games_right,total points_right,total dev points_right,Conference,C_Seed,Conference_right,C_Seed_right
str,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,i64,f64,u32,u32,i64,i64,i64,i64,u32,i64,i64,u32,u32,i64,i64,i64,i64,u32,i64,i64,str,u8,str,u8
"""2021_3104_3268""",2021,3104,3268,null,null,null,null,null,142,100,64,"""N""",0,0.0,16,9,1279,591,1032,719,25,1870,1751,24,2,2215,158,1658,169,26,2373,1827,"""X""",7,"""X""",2
"""2021_3104_3314""",2021,3104,3314,140,80,71,"""N""",0,null,null,null,null,null,1.0,16,9,1279,591,1032,719,25,1870,1751,13,10,1093,628,811,724,23,1721,1535,"""X""",7,"""X""",10
"""2021_3112_3140""",2021,3112,3140,142,52,46,"""N""",0,null,null,null,null,null,1.0,16,5,1114,284,839,338,21,1398,1177,18,5,1320,274,1034,327,23,1594,1361,"""Y""",3,"""Y""",11
"""2021_3112_3163""",2021,3112,3163,151,69,59,"""N""",0,null,null,null,null,null,1.0,16,5,1114,284,839,338,21,1398,1177,24,1,1964,87,1173,90,25,2051,1263,"""Y""",3,"""Z""",1
"""2021_3112_3231""",2021,3112,3231,147,66,53,"""N""",0,null,null,null,null,null,1.0,16,5,1114,284,839,338,21,1398,1177,18,5,1414,337,1033,369,23,1751,1402,"""Y""",3,"""Y""",4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2024_3343_3452""",2024,3343,3452,null,null,null,null,null,138,63,53,"""N""",0,0.0,25,4,1762,260,1347,276,29,2022,1623,24,7,1891,423,1310,481,31,2314,1791,"""Y""",9,"""Y""",8
"""2024_3355_3428""",2024,3355,3428,null,null,null,null,null,138,68,54,"""N""",0,0.0,25,5,1943,270,1451,354,30,2213,1805,20,10,1684,611,1140,735,30,2295,1875,"""X""",12,"""X""",5
"""2024_3394_3425""",2024,3394,3425,null,null,null,null,null,138,87,55,"""H""",0,0.0,19,8,1296,464,1048,560,27,1760,1608,26,5,2007,308,1579,348,31,2315,1927,"""Z""",16,"""Z""",1


In [14]:
male_submission = male_submission.join (male_team_results, how = "left", left_on = ["Season", "Team1"], right_on = ["Season","TeamID"]) 
male_submission = male_submission.join (male_team_results, how = "left", left_on = ["Season", "Team2"], right_on = ["Season","TeamID"]) 
male_submission = male_submission.join (m_seeds_Season, how = "left", left_on = ["Season","Team1"], right_on = ["Season","TeamID"])
male_submission = male_submission.join (m_seeds_Season, how = "left", left_on = ["Season","Team2"], right_on = ["Season","TeamID"])
#male_submission = male_submission.join (m_team_rankings, how = "left", left_on = ["Season","Team1"], right_on = ["Season","TeamID"])
#male_submission = male_submission.join (m_team_rankings, how = "left", left_on = ["Season","Team2"], right_on = ["Season","TeamID"])

print (male_submission.columns)

female_submission = female_submission.join (female_team_results, how = "left", left_on = ["Season", "Team1"], right_on = ["Season","TeamID"]) 
female_submission = female_submission.join (female_team_results, how = "left", left_on = ["Season", "Team2"], right_on = ["Season","TeamID"]) 
female_submission = female_submission.join (w_seeds_Season, how = "left", left_on = ["Season", "Team1"], right_on = ["Season","TeamID"])
female_submission = female_submission.join (w_seeds_Season, how = "left", left_on = ["Season", "Team2"], right_on = ["Season","TeamID"])


print (f" total size {male_training.shape  }   ")
print (f" total size {male_submission.shape }   ")


['ID', 'Season', 'Team1', 'Team2', 'DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT', 'DayNum_right', 'WScore_right', 'LScore_right', 'WLoc_right', 'NumOT_right', 'truth', 'games won', 'games lost', 'points in win', 'points in loss', 'dev points in win', 'dev points in loss', 'total games', 'total points', 'total dev points', 'games won_right', 'games lost_right', 'points in win_right', 'points in loss_right', 'dev points in win_right', 'dev points in loss_right', 'total games_right', 'total points_right', 'total dev points_right', 'Conference', 'C_Seed', 'Conference_right', 'C_Seed_right']
 total size (267, 37)   
 total size (66066, 37)   


In [15]:
['ID', 'Season', 'Team1', 'Team2', 'DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT', 'DayNum_right', 'WScore_right', 'LScore_right', 'WLoc_right', 'NumOT_right', 'truth', 'games won', 'games lost', 'points in win', 'points in loss', 'dev points in win', 'dev points in loss', 'total games', 'total points', 'total dev points', 'games won_right', 'games lost_right', 'points in win_right', 'points in loss_right', 'dev points in win_right', 'dev points in loss_right', 'total games_right', 'total points_right', 'total dev points_right', 'Conference', 'C_Seed', 'Conference_right', 'C_Seed_right']

['ID',
 'Season',
 'Team1',
 'Team2',
 'DayNum',
 'WScore',
 'LScore',
 'WLoc',
 'NumOT',
 'DayNum_right',
 'WScore_right',
 'LScore_right',
 'WLoc_right',
 'NumOT_right',
 'truth',
 'games won',
 'games lost',
 'points in win',
 'points in loss',
 'dev points in win',
 'dev points in loss',
 'total games',
 'total points',
 'total dev points',
 'games won_right',
 'games lost_right',
 'points in win_right',
 'points in loss_right',
 'dev points in win_right',
 'dev points in loss_right',
 'total games_right',
 'total points_right',
 'total dev points_right',
 'Conference',
 'C_Seed',
 'Conference_right',
 'C_Seed_right']

In [16]:
print (male_training.shape) 

print (male_submission.shape)


(267, 37)
(66066, 37)


In [17]:
!pip install ray==2.10.0
!pip install autogluon.tabular --no-cache-dir -q
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.42.1
    Uninstalling ray-2.42.1:
      Successfully uninstalled ray-2.42.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 246.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 205.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 210.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 217.6 MB/s eta 0:00:00


In [18]:

from autogluon.tabular import TabularPredictor
m_predictor = TabularPredictor(path = '/kaggle/working/march_madness/male',
                                       label='truth', 
                               problem_type = 'binary', 
                               eval_metric =  'accuracy',  
                               # sample_weight = 'my_weight',
                               verbosity  = 2,
                               learner_kwargs = {'ignored_columns' : [
                                   'ID']})
                                 
m_predictor.fit(train_data= male_training.to_pandas(), 
                        presets= 'best_quality',
    # best_quality, high_quality, medium_quality, 'experimental_quality',                         
                        time_limit = 16000,
                        num_gpus=0,
                        raise_on_no_models_fitted = True,
                        #dynamic_stacking=False, 
                        #num_stack_levels=0,
                        #hyperparameters=hyper_search,
#                         hyperparameters = my_search_hyperparameters  ,
                        #hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
                        )                              
                                

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.92 GB / 31.35 GB (95.4%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

In [19]:
m_predictor.leaderboard()

 

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_r130_BAG_L1,1.000000,accuracy,0.038783,9.631215,0.038783,9.631215,1,True,31
1,LightGBM_r94_BAG_L1,1.000000,accuracy,0.039606,10.072695,0.039606,10.072695,1,True,61
2,LightGBM_r161_BAG_L1,1.000000,accuracy,0.041157,9.875996,0.041157,9.875996,1,True,40
3,LightGBM_r188_BAG_L1,1.000000,accuracy,0.048116,10.152546,0.048116,10.152546,1,True,27
4,LightGBM_r196_BAG_L1,1.000000,accuracy,0.051450,10.195644,0.051450,10.195644,1,True,44
...,...,...,...,...,...,...,...,...,...,...
215,NeuralNetFastAI_r65_BAG_L2,1.000000,accuracy,1.728635,117.815406,0.481238,32.024958,2,True,176
216,NeuralNetFastAI_r156_BAG_L2,1.000000,accuracy,1.791991,111.612132,0.544595,25.821684,2,True,152
217,ExtraTrees_r4_BAG_L2,1.000000,accuracy,1.885004,87.785127,0.637608,1.994679,2,True,175
218,KNeighborsDist_BAG_L1,0.782772,accuracy,0.002565,0.007023,0.002565,0.007023,1,True,2


In [20]:
w_predictor = TabularPredictor(path = '/kaggle/working/march_madness/female',
                                       label='truth', 
                               problem_type = 'binary', 
                               eval_metric =  'accuracy',  
                               # sample_weight = 'my_weight',
                               verbosity  = 2,
                               learner_kwargs = {'ignored_columns' : [
                                   'ID']})
                                 
w_predictor.fit(train_data= female_training.to_pandas(), 
                        presets= 'best_quality',
    # best_quality, high_quality, medium_quality, 'experimental_quality',                         
                        time_limit = 16000,
                        num_gpus=0,
                        raise_on_no_models_fitted = True,
                        #dynamic_stacking=False, 
                        #num_stack_levels=0,
                        #hyperparameters=hyper_search,
#                         hyperparameters = my_search_hyperparameters  ,
                        #hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
                        )     

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       28.16 GB / 31.35 GB (89.8%)
Disk Space Avail:   19.14 GB / 19.52 GB (98.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked 

In [21]:
w_predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,1.000000,accuracy,0.044991,11.452609,0.044991,11.452609,1,True,3
1,LightGBM_r121_BAG_L1,1.000000,accuracy,0.047482,11.445710,0.047482,11.445710,1,True,87
2,LightGBM_r135_BAG_L1,1.000000,accuracy,0.051280,10.614281,0.051280,10.614281,1,True,82
3,LightGBM_r143_BAG_L1,1.000000,accuracy,0.055360,11.096842,0.055360,11.096842,1,True,57
4,LightGBM_r30_BAG_L1,1.000000,accuracy,0.056514,11.919930,0.056514,11.919930,1,True,69
...,...,...,...,...,...,...,...,...,...,...
215,NeuralNetFastAI_r69_BAG_L1,0.984848,accuracy,0.262797,23.063736,0.262797,23.063736,1,True,84
216,NeuralNetFastAI_r100_BAG_L1,0.984848,accuracy,0.317805,35.593117,0.317805,35.593117,1,True,101
217,NeuralNetFastAI_r65_BAG_L1,0.977273,accuracy,0.470867,31.584669,0.470867,31.584669,1,True,67
218,KNeighborsDist_BAG_L1,0.837121,accuracy,0.003434,0.006405,0.003434,0.006405,1,True,2


In [22]:
schema1 = male_submission.schema
schema2 = male_training.schema    

if schema1 ==schema2 :
    print ("no change detected")
else :
    print ("change detected")

male_submission = male_submission.fill_null(999.0)
male_submission = male_submission.fill_null("W")

male_march_madness_prediction =  m_predictor.predict_proba(male_submission.to_pandas()) 


no change detected


In [23]:
female_submission = female_submission.fill_null(999.0)
female_submission = female_submission.fill_null("W")
female_march_madness_prediction =  w_predictor.predict_proba(female_submission.to_pandas())

In [24]:

march_madness_prediction = pl.concat ([pl.DataFrame(male_march_madness_prediction), pl.DataFrame(female_march_madness_prediction)], how = "vertical")

combined_id = pl.concat ([male_submission.get_column("ID"), female_submission.get_column("ID")], how = "vertical")

print (march_madness_prediction.head(3))

shape: (3, 2)
┌───────┬───────┐
│ 0     ┆ 1     │
│ ---   ┆ ---   │
│ f64   ┆ f64   │
╞═══════╪═══════╡
│ 0.625 ┆ 0.375 │
│ 0.625 ┆ 0.375 │
│ 0.625 ┆ 0.375 │
└───────┴───────┘


In [25]:
probabilty_first_team = march_madness_prediction.get_column ("0")
my_submit = pl.DataFrame ([combined_id, probabilty_first_team]) 
    
my_submit = my_submit.rename ({"0" : "Pred"})

my_submit

ID,Pred
str,f64
"""2025_1101_1102""",0.625
"""2025_1101_1103""",0.625
"""2025_1101_1104""",0.625
"""2025_1101_1105""",0.625
"""2025_1101_1106""",0.625
…,…
"""2025_3477_3479""",0.472214
"""2025_3477_3480""",0.472214
"""2025_3478_3479""",0.472214


In [26]:
my_submit.write_csv("submission.csv")